In [1]:
submit_cv_lines="""#!/usr/bin/bash

#SBATCH --nodes=16
#SBATCH --ntasks=16
#SBATCH --time=02:00:00
#SBATCH --output=./LOG.run.%j.out
#SBATCH --error=./LOG.run.%j.err
#SBATCH --mem-per-cpu=20G
#SBATCH --export=ALL
#SBATCH --cpus-per-task=4
#SBATCH --job-name=cv_mpi_{group}_{set_k}
#SBATCH --wait-all-nodes=1


# either OpenMPI or comparable library
ml OpenMPI

set -x

rm -rf CV

export OMPI_MCA_mpi_warn_on_fork=0
export OMP_NUM_THREADS=${{SLURM_CPUS_PER_TASK}}

srun python -m mpi4py.futures {optim_path}/cv_multi.py \\
     -c {path_root}/{group}/{set_k}/control_file_squash.py \\
     --cv-folder-path {path_root}/{group}/{set_k}/ \\
     --cv-stations /work/yadegari/single_basin_optim_2/stations.json \\
     -l {optim_path}/basin_lut.org \\
     --warmup 365 \\
     --squash-sim \\
     --disable-wbcheck \\
     --htessel-exec {htessel_exec}
     
kge_script_path=/work/yadegari/optim/kge_multiyear.py
for mb in ".";
do
    rm -f $mb/CV/kges.txt && touch $mb/CV/kges.txt
    echo preproc kges for $mb
    for st in $(\ls $mb/CV/ | grep station_);
    do
        echo -n "${{st}} " >> $mb/CV/kges.txt
        python $kge_script_path \\
               -p $mb/CV/$st/default_sim/$st \\
               -w 365 \\
               -n $(echo $st | perl -ne '/.+_(\d+)/; print $1') \\
               >> $mb/CV/kges.txt
    done
done     
"""

In [2]:
optim_path = '/work/yadegari/optim_crash/'
path_root = '/work/yadegari/multibasin_study_1/'

In [3]:
from path import Path

In [4]:
groups = ['with_bucket_system', 'without_bucket_system']
sets = [f'set_{i}' for i in range(1, 11)]

In [5]:
from itertools import product

get_htessel_exec = lambda group: 'master1s_bucket.exe' if group=='with_bucket_system' else 'default'

for set_k, group in product(sets, groups):
    with Path(f'{group}/{set_k}'):
        with open('submit_cv.sh', 'w') as fh:
            fh.writelines(
                submit_cv_lines.format(
                    group=group,
                    set_k=set_k,
                    path_root=path_root,
                    optim_path=optim_path,
                    htessel_exec=get_htessel_exec(group)
                )
            )

In [6]:
import subprocess as sp

for group in groups:
    for set_k in sets:
        with Path(f'{group}/{set_k}'):
            sp.Popen('rm -rf CV', shell=True).communicate()
            sp.Popen('sbatch submit_cv.sh', shell=True).communicate()